In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# 1. Configuración de Spark (Misma estructura que tu archivo de pedidos)
packages = [
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0",
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "com.amazonaws:aws-java-sdk-bundle:1.12.262"
]

spark = SparkSession.builder \
    .appName("AlertasTemperaturaStreaming") \
    .config("spark.jars.packages", ",".join(packages)) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [2]:
from pyspark.sql.functions import from_json, col, to_timestamp
from pyspark.sql.types import *

# 1. Esquema exacto para tus datos JSON
# Coincide con: {"sensor_id": "TEMP_01", "temperature": 20.77, "timestamp": "...", "location": "..."}
schema_temp = StructType([
    StructField("sensor_id", StringType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("timestamp", StringType(), True),
    StructField("location", StringType(), True)
])

# 2. Lectura de Kafka
kafka_temp_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "temperature-sensors") \
    .option("startingOffsets", "latest") \
    .load()

# 3. Parseo de JSON (Estilo profesional de tu clase)
parsed_temp_df = kafka_temp_df.select(
    col("timestamp").alias("kafka_arrival_time"),
    from_json(col("value").cast("string"), schema_temp).alias("data")
).select(
    "kafka_arrival_time",
    "data.sensor_id",
    "data.temperature",
    col("data.timestamp").alias("sensor_timestamp"),
    "data.location"
).withColumn("event_timestamp", to_timestamp(col("sensor_timestamp")))

# 4. Función de Alertas (Para que se vea claro en pantalla)
def procesar_alertas(batch_df, batch_id):
    # Filtramos las temperaturas altas (> 25)
    alertas = batch_df.filter(col("temperature") > 25.0)
    
    if not alertas.isEmpty():
        print(f"\n🔥 --- ¡ALERTA DE ALTA TEMPERATURA! (Batch: {batch_id}) ---")
        print(f"Se han detectado {alertas.count()} lecturas críticas:")
        # Mostramos los detalles de la alerta
        alertas.select("sensor_id", "temperature", "location", "sensor_timestamp").show(truncate=False)
    else:
        # Esto indica que el sistema funciona pero las temperaturas son normales
        if not batch_df.isEmpty():
            print(f"✅ Batch {batch_id}: {batch_df.count()} lecturas recibidas. Todo bajo control (<= 25°C).")

# 5. Inicio del Stream con Trigger de 5 segundos
query_alertas = parsed_temp_df.writeStream \
    .foreachBatch(procesar_alertas) \
    .trigger(processingTime="5 seconds") \
    .start()

print("🚀 Sistema de Alertas iniciado en SEGUNDO PLANO.")
print("Puedes seguir usando el cuaderno. Las alertas aparecerán aquí abajo.")

🚀 Sistema de Alertas iniciado en SEGUNDO PLANO.
Puedes seguir usando el cuaderno. Las alertas aparecerán aquí abajo.
✅ Batch 1: 1 lecturas recibidas. Todo bajo control (<= 25°C).

🔥 --- ¡ALERTA DE ALTA TEMPERATURA! (Batch: 2) ---
Se han detectado 1 lecturas críticas:
+---------+-----------+-----------+--------------------------------+
|sensor_id|temperature|location   |sensor_timestamp                |
+---------+-----------+-----------+--------------------------------+
|TEMP_02  |26.32      |Warehouse_B|2026-02-19T12:01:01.372122+00:00|
+---------+-----------+-----------+--------------------------------+


🔥 --- ¡ALERTA DE ALTA TEMPERATURA! (Batch: 3) ---
Se han detectado 1 lecturas críticas:
+---------+-----------+-----------+--------------------------------+
|sensor_id|temperature|location   |sensor_timestamp                |
+---------+-----------+-----------+--------------------------------+
|TEMP_02  |26.06      |Warehouse_B|2026-02-19T12:01:06.372489+00:00|
+---------+----------

In [3]:
# Detener TODOS los streams activos de golpe
for query in spark.streams.active:
    print(f"Deteniendo query: {query.name} (ID: {query.id})")
    query.stop()

print("🛑 Todos los procesos detenidos.")

Deteniendo query: None (ID: 559a426f-66ca-4882-881b-d2bf4a53caf6)
🛑 Todos los procesos detenidos.
